In [48]:
from pyspark.sql import SparkSession
import getpass
from pyspark.sql.functions import col, sha2, concat_ws


user_name = getpass.getuser()

spark = SparkSession.builder \
        .config('spark.ui.port',0) \
        .config('spark.sql.warehouse.dir', f"/user/{user_name}/warehouse") \
        .config('spark.shuffle.useOldFetchProtocol', 'true') \
        .enableHiveSupport() \
        .master('yarn') \
        .appName('SRV_Main') \
        .getOrCreate()

In [49]:
%run "/home/itv004483/Reporting.ipynb"
logpath = f"/home/{user_name}/Reporting/SRV/report_{dts}.txt"
raw_path = '/public/trendytech/datasets/accepted_2007_to_2018Q4.csv'

/home/itv004483/Reporting/report_2024-01-07_22:08:08.txt
2024-01-07


In [38]:
addLog(f"Job executed by user - {user_name}")
addLog("Job Started at : "+str(dts))
addLog("")

In [39]:
addSparkOptions(spark)

In [51]:
DF = spark.read \
        .format("csv") \
        .option("header","true") \
        .option("path",raw_path) \
        .load()

rawDF = DF.withColumn("name_sha2", sha2(concat_ws('||',*['emp_title','emp_length','home_ownership','annual_inc','zip_code','addr_state','grade','sub_grade','verification_status'])
                                        ,256))

+--------------------+
|           name_sha2|
+--------------------+
|6d5091b3fcaaeb4ea...|
|b5e7938b0a2da4cea...|
|91060b858433e8a61...|
|cab1fa9f533688b0a...|
|f74e401c1ab0adf78...|
|8aef4bb29d609d8d6...|
|538b4653da3b1e814...|
|b24d55f21390533c5...|
|1035c5401b0ca76d0...|
|cb0f1777593e77909...|
|a962f4d59caec5fa1...|
|e7592ab57b3afd9f1...|
|603afa9d1be879b7b...|
|9fe2d59ddf2a4f37e...|
|23857480ccf555ce4...|
|7c69d5f36fdabf6e1...|
|08bf9e080503b0113...|
|1cafb05aa6c894c30...|
|f2c4010f700d8c9c4...|
|05ad4aed7c393035e...|
+--------------------+
only showing top 20 rows



In [52]:
#run customers srv
%run "/home/itv004483/LendingClub/SRV/SRV_Customers.ipynb"
srv_cust_stat = cust_stat
if(srv_cust_stat == "Success"):
    print("SRV Customers completed successfully")
else:
    print("SRV Customers failed")

SRV Customers completed successfully


In [42]:
#run Loans srv
%run "/home/itv004483/LendingClub/SRV/SRV_Loans.ipynb"
srv_loans_stat = loans_stat
if(srv_loans_stat == "Success"):
    print("SRV Loans completed successfully")
else:
    print("SRV Loans failed")

/user/itv004483/lendingClub/srv/2024-01-07/loans/
SRV Loans completed successfully


In [43]:
#run loan_repayments srv
%run "/home/itv004483/LendingClub/SRV/SRV_LoanRepayments.ipynb"
srv_loans_rp_stat = lnrp_stat
if(srv_loans_rp_stat == "Success"):
    print("SRV loan_repayments completed successfully")
else:
    print("SRV loan_repayments failed")

/user/itv004483/lendingClub/srv/2024-01-07/loan_repayments/
SRV loan_repayments completed successfully


In [53]:
#run loan defaulters srv
%run "/home/itv004483/LendingClub/SRV/SRV_LoanDefaulters.ipynb"
srv_loans_df_stat = lndf_stat
if(srv_loans_df_stat == "Success"):
    print("SRV loan_defaulters completed successfully")
else:
    print("SRV loan_defaulters failed")

/user/itv004483/lendingClub/srv/2024-01-07/loan_defaulters/
SRV loan_defaulters completed successfully


In [45]:
endtime = dt.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
addLog("")
addLog("Job end time " + str(endtime))
addLog("")


In [46]:
saveLog(logpath,logs)


In [54]:
spark.stop()